# Se importan las bibliotecas necesarias

In [71]:
import pandas as pd
import re
import xgboost as xgb
import matplotlib as plt
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score

# Se lee el .csv

In [2]:
train = pd.read_csv('csv/train.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [3]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


# Elaboración del modelo

Para este modelo se utilizará XGBoost para realizar las predicciones y se tendrán en cuenta los siguientes features extraídos del análisis exploratorio, los cuales fueron seleccionados por el grupo como aquellos que podrían resultan más interesantes:

- Palabras con mayor y menor porcentaje de veracidad.
- Pares de 2 palabras con mayor y menor porcentaje de veracidad.
- Tweets que contienen @, ¿?, ¡! tienden a ser falsos.
- Keywords con mayor y menor porcentaje de veracidad.
- Locaciones más y menos veraces.
- Longitud.

### Preparación del set de datos

#### Parte inicial

In [4]:
#Palabras con mayor y menor porcentaje de veracidad
min_repetitions = ((0.2/100)*len(train.index))
words = train['text'].str.split()
clean_words = []

for sentence in words:
    clean_sentence = []
    for word in sentence:
        clean_word = re.sub('[^A-Za-z0-9]+','', word)
        clean_sentence.append(clean_word.lower())
    clean_words.append(clean_sentence)

In [5]:
train_aux = train
train_aux['words'] = clean_words
train_aux = train_aux.explode('words')
train_aux.head()

,id,keyword,location,text,target,words
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deeds
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,are
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,the
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,reason


In [6]:
train_aux = train_aux.groupby('words').agg({'target':['sum','count']})
train_aux = train_aux[train_aux['target']['count']>min_repetitions]
train_aux['veracity'] = train_aux['target']['sum']/train_aux['target']['count']
train_aux.head()

target        veracity
         sum count          
words                       
         904  2150  0.420465
05        14    18  0.777778
1         29    64  0.453125
10         9    23  0.391304
100        6    17  0.352941

In [7]:
train_aux.reset_index(inplace=True) 

In [8]:
#Pares de 2 palabras con mayor y menor porcentaje de veracidad.
train_aux2 = train
train_aux2['words'] = clean_words
words_pairs = []

for sentence in train_aux2['words']:
    pairs = []
    for i in range(len(sentence)-1):
        pairs.append(sentence[i] + ' ' + sentence[i+1])
    words_pairs.append(pairs)
    
train_aux2['words'] = words_pairs
train_aux2 = train_aux2.explode('words')
train_aux2.head()

,id,keyword,location,text,target,words
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deeds are
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,are the
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,the reason
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,reason of


In [9]:
train_aux2 = train_aux2.groupby('words').agg({'target':['sum','count']})
train_aux2 = train_aux2[train_aux2['target']['count']>min_repetitions]
train_aux2['veracity'] = train_aux2['target']['sum']/train_aux2['target']['count']

In [10]:
#las palabras que son '' la cagan
train_aux2.head(10)

target        veracity
                 sum count          
words                               
                  66   118  0.559322
 abc              19    19  1.000000
 full              6    67  0.089552
 i                 6    19  0.315789
 the              32    57  0.561404
11yearold boy     20    20  1.000000
16yr old          28    28  1.000000
40 families       26    26  1.000000
70 years          30    31  0.967742
a blast            3    16  0.187500

In [11]:
train_aux2.reset_index(inplace=True)

In [12]:
# Keywords más y menos veraces
train_aux3 = train
train_aux3 = train_aux3.groupby('keyword').agg({'target':['sum','count']})
train_aux3 = train_aux3[train_aux3['target']['count']>min_repetitions]
train_aux3['veracity'] = train_aux3['target']['sum']/train_aux3['target']['count']
train_aux3.head()

target        veracity
                       sum count          
keyword                                   
ablaze                  13    36  0.361111
accident                24    35  0.685714
aftershock               0    34  0.000000
airplane%20accident     30    35  0.857143
ambulance               20    38  0.526316

In [13]:
train_aux3.reset_index(inplace=True)

In [14]:
# Locaciones más y menos veraces
train_aux4 = train
train_aux4 = train_aux4.groupby('location').agg({'target':['sum','count']})
train_aux4 = train_aux4[train_aux4['target']['count']>min_repetitions]
train_aux4['veracity'] = train_aux4['target']['sum']/train_aux4['target']['count']
train_aux4.head()

target        veracity
               sum count          
location                          
Australia        9    18  0.500000
California       7    17  0.411765
Canada          13    29  0.448276
Chicago, IL      9    18  0.500000
India           20    24  0.833333

In [15]:
train_aux4.reset_index(inplace=True)

#### Parte final

In [16]:
top_words = train_aux[train_aux['veracity']>0.9]['words']
worst_words = train_aux[train_aux['veracity']<0.1]['words']

In [17]:
top_words_pairs = train_aux2[train_aux2['veracity']>0.9]['words']
worst_words_pairs = train_aux2[train_aux2['veracity']<0.1]['words']

In [18]:
top_keywords = train_aux3[train_aux3['veracity']>0.9]['keyword']
worst_keywords = train_aux3[train_aux3['veracity']<0.1]['keyword']

In [19]:
top_locations = train_aux4[train_aux4['veracity']>0.9]['location']
worst_locations = train_aux4[train_aux4['veracity']<0.1]['location']

In [20]:
train = train.drop(columns='words')
train['long(char)'] = train['text'].str.len()

aux = []
for i in words:
    aux.append(len(i))

train['long(word)'] = aux
train.head()

,id,keyword,location,text,target,long(char),long(word)
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,13
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,7
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,22
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,8
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,16


In [21]:
def count_text_ocurrences(texts, words):
    ocurrences = []
    for text in texts:
        count = 0
        for word in words:
            if word in text:
                count = count+1
        ocurrences.append(count)
    return ocurrences

def count_ocurrences(searched, series):
    ocurrences = []
    for element in searched:
        count = 0
        if element in series:
            count = count+1
        ocurrences.append(count)
    return ocurrences

def contains_char(texts, char1, char2=''):
    ocurrences = []
    for text in texts:
        count = 0
        if char1 in text:
            count = 1
        if (count==0)&(char2!=''):
            if char2 in text:
                count = 1
        ocurrences.append(count)
    return ocurrences

In [22]:
train['top_words'] = count_text_ocurrences(train['text'], top_words)
train['worst_words'] = count_text_ocurrences(train['text'], worst_words)
train['top_words_pairs'] = count_text_ocurrences(train['text'], top_words_pairs)
train['worst_words_pairs'] = count_text_ocurrences(train['text'], worst_words_pairs)
train.head()

,id,keyword,location,text,target,long(char),long(word),top_words,worst_words,top_words_pairs,worst_words_pairs
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,13,0,0,0,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,7,0,0,0,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,22,1,0,0,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,8,1,0,0,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,16,1,0,0,0


In [23]:
train['top_keywords'] = count_ocurrences(train['keyword'], top_keywords)
train['worst_keywords'] = count_ocurrences(train['keyword'], worst_keywords)
train['top_locations'] = count_ocurrences(train['location'], top_locations)
train['worst_locations'] = count_ocurrences(train['location'], worst_locations)
train.head()

,id,keyword,location,text,target,long(char),long(word),top_words,worst_words,top_words_pairs,worst_words_pairs,top_keywords,worst_keywords,top_locations,worst_locations
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,13,0,0,0,0,0,0,0,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,7,0,0,0,0,0,0,0,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,22,1,0,0,0,0,0,0,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,8,1,0,0,0,0,0,0,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,16,1,0,0,0,0,0,0,0


In [24]:
train['@'] = contains_char(train['text'],'@')
train['¿,?'] = contains_char(train['text'],'\?', '¿')
train['!,¡'] = contains_char(train['text'],'!','¡')
train.head()

,id,keyword,location,text,target,long(char),long(word),top_words,worst_words,top_words_pairs,worst_words_pairs,top_keywords,worst_keywords,top_locations,worst_locations,@,"¿,?","!,¡"
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,13,0,0,0,0,0,0,0,0,0,0,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,7,0,0,0,0,0,0,0,0,0,0,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,22,1,0,0,0,0,0,0,0,0,0,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,8,1,0,0,0,0,0,0,0,0,0,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,16,1,0,0,0,0,0,0,0,0,0,0


In [25]:
train = train.drop(columns=['keyword','location','text'])
train = train.set_index('id')
train.head()

,target,long(char),long(word),top_words,worst_words,top_words_pairs,worst_words_pairs,top_keywords,worst_keywords,top_locations,worst_locations,@,"¿,?","!,¡"
id,,,,,,,,,,,,,,
1,1,69,13,0,0,0,0,0,0,0,0,0,0,0
4,1,38,7,0,0,0,0,0,0,0,0,0,0,0
5,1,133,22,1,0,0,0,0,0,0,0,0,0,0
6,1,65,8,1,0,0,0,0,0,0,0,0,0,0
7,1,88,16,1,0,0,0,0,0,0,0,0,0,0


### Entrenamiento del set de datos

In [52]:
X, y = train.iloc[:,1:-1], train.iloc[:,0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=131)

In [66]:
xg_reg = xgb.XGBRegressor(objective ='binary:logistic', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 20, alpha = 10, n_estimators = 80)

In [67]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=20,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=80, n_jobs=0, num_parallel_tree=1,
             objective='binary:logistic', random_state=0, reg_alpha=10,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [68]:
preds = xg_reg.predict(X_test)

In [69]:
print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

Accuracy Score : 0.7162872154115587
Precision Score : 0.6890080428954424
Recall Score : 0.5526881720430108
F1 Score : 0.613365155131265


In [75]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }
grid_acc = GridSearchCV(xg_reg, param_grid = params,scoring = 'recall')
grid_acc.fit(X_train, y_train)

ValueError: Classification metrics can't handle a mix of binary and continuous targets